In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import re
from time import gmtime, strftime
import sys
import os
import io

import zipfile

In [5]:
with zipfile.ZipFile("data/full-oldb.20220708.zip", 'r') as zip_ref:
    zip_ref.extractall("data/original")

In [6]:
# df = pd.read_csv('data/edited/nodes-addresses_v2.csv')
df = pd.read_csv('data/original/nodes-addresses.csv')

df.head()

/Users/julie.fisher/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (2,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,node_id,address,name,countries,country_codes,sourceID,valid_until,note
0,24000001,"ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
1,24000002,"SUITE E-2,UNION COURT BUILDING, P.O. BOX N-818...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
2,24000003,"LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785,...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
3,24000004,"P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O....",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
4,24000005,"LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. ...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN


## Move addresses in name col to address col

Some of the entries have the address incorrectly in the name column

In [7]:
df.loc[df['name'].str.len() > df['address'].str.len(), ['address', 'name']]

,address,name
528,6B Chenyu Court,6B Chenyu Court; 22-24 Kennedy Road; Hong Kong
529,15C Suchun Industrial Square,15C Suchun Industrial Square; Suzhou Industria...
151607,"8F., No. 68","8F., No. 68; Minfu 13th St. Taoyuan County; 33..."
151608,"No. 66, Ln. 20","No. 66, Ln. 20; Dafu Rd., Shengang Township; 4..."
151609,"1F., No. 18, Ln. 274","1F., No. 18, Ln. 274; Chang'an St., Luzhou Cit..."
...,...,...
210828,"285 Central Park West, #2","285 Central Park West, #2; New York; New York ..."
210829,9481 Gatetrail Drive,9481 Gatetrail Drive; Dallas; Texas 75238; Uni...
210830,2822 Millers Run Lane,2822 Millers Run Lane; Lancaster; PA 17601; Un...
210831,24 Bullimore Grove,24 Bullimore Grove; Kenilworth CV8 2QF; Warwic...


In [8]:
df.loc[df['name'].str.len() > df['address'].str.len(), ['address', 'name']
      ] = df.loc[df['name'].str.len() > df['address'].str.len(), ['name', 'address']].values

In [9]:
df.loc[df['name'].notna() & df['address'].isna()]

,node_id,address,name,countries,country_codes,sourceID,valid_until,note
210833,88000000,NaN,"WINDWARD SIDE , , SABA",NaN,NaN,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,NaN
210834,88000001,NaN,"PISO 3, PUERTA 4, ANDORRA LA VELLA , PRINCIPAT...",Andorra,AND,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,NaN
210835,88000002,NaN,"BLOWING POINT , CUL DE SAC, ANGUILLA",Anguilla,AIA,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,NaN
210836,88000003,NaN,"CARIBBEAN COMMERCIAL CENTER, POSTBUS 20, ANGUI...",Anguilla,AIA,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,NaN
210837,88000004,NaN,"HEYWOOD HOUSE SOUTH HILL, ANGUILLA, BRITISH WE...",Anguilla,AIA,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,NaN
...,...,...,...,...,...,...,...,...
390990,68001690,NaN,"Manase, Samoa",Samoa,WSM,Paradise Papers - Samoa corporate registry,Samoa corporate registry data is current throu...,NaN
390991,68001691,NaN,"Vailele-uta, Faleata Sisifo, Samoa",Samoa,WSM,Paradise Papers - Samoa corporate registry,Samoa corporate registry data is current throu...,NaN
390992,68001692,NaN,"Lotopa, Falelatai & Samatau, Samoa",Samoa,WSM,Paradise Papers - Samoa corporate registry,Samoa corporate registry data is current throu...,NaN
390993,68001693,NaN,"Sogi, Vaisigano No.2, Samoa",Samoa,WSM,Paradise Papers - Samoa corporate registry,Samoa corporate registry data is current throu...,NaN


In [10]:
df.loc[df['name'].notna() & df['address'].isna(), ['address', 'name']
      ] = df.loc[df['name'].notna() & df['address'].isna(), ['name', 'address']].values

## Country name in countries col

The country_code snuck into some of the values for the countries column

In [11]:
df[df['countries'].str.len()<=3]

,node_id,address,name,countries,country_codes,sourceID,valid_until,note
396222,240450001,"31/F., CHINACHEM CENTURY TOWER, 178 GLOUCESTER...",NaN,HKG,HKG,Pandora Papers - SFM Corporate Services,Provider data is current through 2014,NaN
396223,240450003,"UNIT 117, ORION MALL, PALM STREET, P.0. BOX 82...",NaN,SYC,SYC,Pandora Papers - SFM Corporate Services,Provider data is current through 2019,NaN
396224,240450004,"P.0O. BOX 3175, ROAD TOWN, TORTOLA, BRITISH VI...",NaN,VGB,VGB,Pandora Papers - SFM Corporate Services,Provider data is current through 2015,NaN
396225,240450005,"UNIT 117, ORION MALL, PALM STREET, P.O. BOX 82...",NaN,SYC,SYC,Pandora Papers - SFM Corporate Services,Provider data is current through 2014,NaN
396226,240450007,"1 ½ MILES NORTHERN HIGHWAY, BELIZE CITY, BELIZE",NaN,BLZ,BLZ,Pandora Papers - SFM Corporate Services,Provider data is current through 2014,NaN
...,...,...,...,...,...,...,...,...
400697,240453574,"C 49 SUR 5106, ESTRELLA DEL SUR, 72190 PUEBLA,...",NaN,MEX,MEX,Pandora Papers - SFM Corporate Services,Provider data is current through 2016,NaN
400699,240453575,"C DA PONZANELLO MAMURRANO, 99997/SNC, COMUNE D...",NaN,ITA,ITA,Pandora Papers - SFM Corporate Services,Provider data is current through 2012,NaN
400701,240453576,"STEENOKKERZEELSTRAAT 78, 1930 ZAVENTEM, BELGIUM",NaN,BEL,BEL,Pandora Papers - SFM Corporate Services,Provider data is current through 2013,NaN
400703,240453577,"M. NIKITSKAYA STREET, 1615 AP. 155, 121069 MOS...",NaN,RUS,RUS,Pandora Papers - SFM Corporate Services,Provider data is current through 2014,NaN


In [12]:
df.loc[df['countries'].str.len()<=3, 'countries'].value_counts()

GBR    222
ITA    200
ESP    156
DEU     94
FRA     69
      ... 
VUT      1
AND      1
MDG      1
JEY      1
LTU      1
Name: countries, Length: 123, dtype: int64

In [13]:
codes_to_countries = pd.DataFrame(
    df.loc[
        df['countries'].str.len()>3
    ].groupby('country_codes')['countries'
                              ].value_counts()).drop(['countries'], axis=1
                                                    ).reset_index().set_index('country_codes').to_dict()['countries']
codes_to_countries

{'ABW': 'Aruba',
 'AFG': 'Afghanistan',
 'AGO': 'Angola',
 'AIA': 'Anguilla',
 'ALB': 'Albania',
 'AND': 'Andorra',
 'ANT': 'Netherlands Antilles',
 'ARE': 'United Arab Emirates',
 'ARG': 'Argentina',
 'ARM': 'Armenia',
 'ASM': 'American Samoa',
 'ATG': 'Antigua and Barbuda',
 'AUS': 'Australia',
 'AUT': 'Austria',
 'AUT;LIE': 'Austria;Liechtenstein',
 'AZE': 'Azerbaijan',
 'BEL': 'Belgium',
 'BEN': 'Benin',
 'BES': 'Sint Eustatius and Saba Bonaire',
 'BFA': 'Burkina Faso',
 'BGD': 'Bangladesh',
 'BGR': 'Bulgaria',
 'BHR': 'Bahrain',
 'BHS': 'Bahamas',
 'BIH': 'Bosnia and Herzegovina',
 'BLR': 'Belarus',
 'BLZ': 'Belize',
 'BMU': 'Bermuda',
 'BOL': 'Plurinational State of Bolivia',
 'BRA': 'Brazil',
 'BRB': 'Barbados',
 'BRN': 'Brunei Darussalam',
 'BTN': 'Bhutan',
 'BWA': 'Botswana',
 'CAF': 'Central African Republic',
 'CAN': 'Canada',
 'CHE': 'Switzerland',
 'CHL': 'Chile',
 'CHN': 'China',
 'CIV': "Côte d'Ivoire",
 'CMR': 'Cameroon',
 'COD': 'The Democratic Republic of the Congo',


In [14]:
df['countries'].value_counts(dropna=False)

NaN                                         125327
China                                        32491
Hong Kong                                    28255
United States                                24349
Barbados                                     19348
                                             ...  
Syrian Arab Republic                             1
LAO                                              1
Bhutan                                           1
VUT                                              1
British Virgin Islands;Brunei Darussalam         1
Name: countries, Length: 366, dtype: int64

In [15]:
df['country_codes'].value_counts(dropna=False)

NaN        125327
CHN         32491
HKG         28269
USA         24365
BRB         19348
            ...  
TUV             1
BTN             1
IMN;GBR         1
AUT;LIE         1
VGB;BRN         1
Name: country_codes, Length: 226, dtype: int64

In [16]:
df.loc[df['countries'].str.len()<=3, 'countries'] = df.loc[df['countries'].str.len()<=3, 'countries'].map(codes_to_countries)

In [17]:
df[df['countries'].str.len()<=3]

,node_id,address,name,countries,country_codes,sourceID,valid_until,note


In [18]:
df['countries'].value_counts(dropna=False)

NaN                                         125327
China                                        32491
Hong Kong                                    28269
United States                                24365
Barbados                                     19348
                                             ...  
New Caledonia                                    1
Virgin Islands, U.S.                             1
Northern Mariana Islands                         1
Bhutan                                           1
British Virgin Islands;Brunei Darussalam         1
Name: countries, Length: 243, dtype: int64

In [19]:
df['country_codes'].value_counts(dropna=False)

NaN        125327
CHN         32491
HKG         28269
USA         24365
BRB         19348
            ...  
TUV             1
BTN             1
IMN;GBR         1
AUT;LIE         1
VGB;BRN         1
Name: country_codes, Length: 226, dtype: int64

## Break into more managable smaller pieces

In [20]:
country_cts = pd.DataFrame(df['countries'].value_counts(dropna=False)).reset_index()
country_cts.columns = ['country', 'count']
country_cts

,country,count
0,NaN,125327
1,China,32491
2,Hong Kong,28269
3,United States,24365
4,Barbados,19348
...,...,...
238,New Caledonia,1
239,"Virgin Islands, U.S.",1
240,Northern Mariana Islands,1
241,Bhutan,1


In [21]:
country_cts[country_cts['count'] < 500]

,country,count
66,Malta,473
67,Sweden,422
68,Costa Rica,417
69,Czech Republic,412
70,Kazakhstan,411
...,...,...
238,New Caledonia,1
239,"Virgin Islands, U.S.",1
240,Northern Mariana Islands,1
241,Bhutan,1


In [22]:
!mkdir data/edited

In [23]:
df[df['countries'].isin(
    country_cts[country_cts['count'] < 500][
        'country'].to_list())].to_csv('data/edited/addresses_low_representation.csv', index=False)

In [24]:
for country in country_cts[country_cts['count'] >= 500]['country'].to_list():
    df[df['countries'] == country].to_csv('data/edited/addresses_{}.csv'.format(country), index=False)